<h1>Linjär reggression</h1>
<h2>Analys av california housing dataset</h2>  
<em>Adrian Söderberg Skog</em><br>
<strong>AIMG25</strong>
<h3>Importer</h3>
<p></p>

In [92]:
import numpy as np
import pandas as pd
import LinearRegression as lreg
import importlib
importlib.reload(lreg)

<module 'LinearRegression' from 'c:\\Users\\adria\\Desktop\\Python\\Statistiska_Metoder\\Labb\\LinearRegression.py'>

<h3>Ladda in och inspektera dataset</h3>

In [93]:
df = pd.read_csv("housing.csv")
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [95]:
df.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

<h3>Bearbetning av data</h3>
<p>Datasetet innehåller 207 NaN värden i kolumnen "total bedrooms". Eftersom det är en liten del av de totala antalet rader på 20640, gör jag bedömningen att det är relativt säkert att ta bort dessa. </p>

In [96]:
df_clean = df.dropna()



<p>Vi kan också se att det finns en kolumn med kategoriska värden. För att kunna inkludera dessa i reggressionen behöver de kodas om till numeriska värden. Eftersom de inte har någon tydlig hirarkisk struktur gör jag bedömningen att det lämpligaste är att använda one hot encoding. I min kod fil har jag därför också gjort en klass oneHotEncoding som kan omvandla den kategoriska kolumnen till nya kolumner med binära värden. </p>

In [97]:
encoder = lreg.OneHotEncoder()

y = df_clean.iloc[:,8].to_numpy()
X = pd.concat([df_clean.iloc[:,:8], df_clean.iloc[:,-1]], axis=1) 

X_enc = encoder.fit_transform(X, column_names=df_clean.columns)

<p>Sista steget innan vi kan göra en linjär regressionsanalys på datasetet är att det behöver delas upp i träningsdata och testdata samt att målvariabeln behöver separeras från de övriga parametrarna. I kodfilen har jag gort en funktion som slumpmässigt delar upp punkterna. Defaultuppdelningen är 80% i träningsdata och 20% i testdata. </p>
<p>Målvariabeln kommer att vara median house value, eftersom det är huspriser vi vill kunna förutsäga med hjälp av de andra parametrarna. </p>

In [98]:
data = np.column_stack((X_enc, y))

train_Data, test_Data = lreg.train_test_split(data)

train_X, train_y = train_Data[:,:-1], train_Data[:,-1]
test_X, test_y = test_Data[:,:-1], test_Data[:,-1]

feature_names = encoder.feature_names_

In [99]:
model = lreg.LinearRegression()
model.fit(train_X, train_y)

(array([-2.79174059e+04, -2.66617373e+04,  1.03322678e+03, -5.31296737e+00,
         9.29026132e+01, -4.32248980e+01,  6.50785115e+01,  3.90231690e+04,
        -3.86939518e+04,  1.51344513e+05, -4.99515729e+03,  2.34775400e+03]),
 np.float64(-2356380.6642318955))

In [102]:
model.summary(feature_names=feature_names, target_name="median house value", testX= test_X, testy= test_y, CL=0.95)

Linear regression summary
Target variable:            median house value
Sample size:                16346
Number of features:         12

variance:                   4711408820.1778
Standard deviation:         68639.7029
R square:                   0.6480
Adjusted R square:          0.6477
test RMSE:                  68960.5997
F-statistic:                2505.3060
p value (F-statistic):      0.0000

Confidence level:   95.0%
Feature                 CI lower   coefficient   CI upper  t-value   p-value   significant
longitude                  -30161.05 -27917.41 -25673.77    -24.39      0.00       Yes
latitude                   -28874.07 -26661.74 -24449.40    -23.62      0.00       Yes
housing_median_age            937.23   1033.23   1129.23     21.10      0.00       Yes
total_rooms                    -7.05     -5.31     -3.58     -6.01      0.00       Yes
total_bedrooms                 77.95     92.90    107.86     12.17      0.00       Yes
population                    -45.74    -43

In [101]:
names = feature_names + ["median house value"]
model.corr_matrix(data, column_names=names)


Pearson correlation:

longitude                  [' 1.00', '-0.92', '-0.11', ' 0.05', ' 0.07', ' 0.10', ' 0.06', '-0.02', '-0.06', ' 0.01', '-0.47', ' 0.05', '-0.05']
latitude                   ['-0.92', ' 1.00', ' 0.01', '-0.04', '-0.07', '-0.11', '-0.07', '-0.08', ' 0.35', '-0.02', ' 0.36', '-0.16', '-0.14']
housing_median_age         ['-0.11', ' 0.01', ' 1.00', '-0.36', '-0.32', '-0.30', '-0.30', '-0.12', '-0.24', ' 0.02', ' 0.26', ' 0.02', ' 0.11']
total_rooms                [' 0.05', '-0.04', '-0.36', ' 1.00', ' 0.93', ' 0.86', ' 0.92', ' 0.20', ' 0.03', '-0.01', '-0.02', '-0.01', ' 0.13']
total_bedrooms             [' 0.07', '-0.07', '-0.32', ' 0.93', ' 1.00', ' 0.88', ' 0.98', '-0.01', '-0.01', '-0.00', '-0.02', ' 0.00', ' 0.05']
population                 [' 0.10', '-0.11', '-0.30', ' 0.86', ' 0.88', ' 1.00', ' 0.91', ' 0.01', '-0.02', '-0.01', '-0.06', '-0.02', '-0.03']
households                 [' 0.06', '-0.07', '-0.30', ' 0.92', ' 0.98', ' 0.91', ' 1.00', ' 0.01', '-0.04',